Import Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from pathlib import Path

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import StandardScaler, LabelEncoder



In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
url = "/content/gdrive/My Drive/Uni/Bootcamp/Bootcamp1/Project_4/winequality-red.csv"
rchem_df = pd.read_csv(url)
rchem_df.head()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
rchem_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [5]:
#Create bins for good and bad
from sklearn.preprocessing import LabelEncoder
bins = (2, 6.5, 8)
group_names = ['bad', 'good']
rchem_df['quality'] = pd.cut(rchem_df['quality'], bins = bins, labels = group_names)
label_quality = LabelEncoder()
rchem_df['quality'] = label_quality.fit_transform(rchem_df['quality'])
rchem_df['quality'].value_counts()

0    1382
1     217
Name: quality, dtype: int64

In [6]:
from re import X
X = rchem_df.drop('quality',axis=1)
y=rchem_df['quality']
X.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


In [7]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: quality, dtype: int64

In [8]:
rchem_df.columns[:11]

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol'],
      dtype='object')

Create a model

In [9]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=10)

Some large variation in numbers so we will scale the data

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train2 = pd.DataFrame(sc.fit_transform(X_train))
X_test2 = pd.DataFrame(sc.transform(X_test))
X_train2.columns = X_train.columns.values
X_test2.columns = X_test.columns.values
X_train2.index = X_train.index.values
X_test2.index = X_test.index.values
X_train = X_train2
X_test = X_test2

In [11]:
## Logistic Regression
from sklearn.linear_model import LogisticRegression
lr_classifier = LogisticRegression(random_state = 42)
lr_classifier.fit(X_train, y_train)

# Predicting Test Set
y_pred = lr_classifier.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

results = pd.DataFrame([['Logistic Regression', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])
print(results)
print('Logistic Regression Training Accuracy : ',
      metrics.accuracy_score(y_train,
                             lr_classifier.predict(X_train))*100)
print('Logistics Regression Validation Accuracy : ',
      metrics.accuracy_score(y_test,
                             lr_classifier.predict(X_test))*100)

                 Model  Accuracy  Precision    Recall  F1 Score
0  Logistic Regression     0.875   0.615385  0.347826  0.444444
Logistic Regression Training Accuracy :  88.65058087578194
Logistics Regression Validation Accuracy :  87.5


In [12]:
## SVM (Support Vector Machines) using Support Vector Classifier
from sklearn.svm import SVC
svc_classifier = SVC(random_state = 7, kernel = 'linear')
svc_classifier.fit(X_train, y_train)

# Predicting Test Set
y_pred = svc_classifier.predict(X_test)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

model_results = pd.DataFrame([['SVM (Linear)', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

results = results.append(model_results, ignore_index = True)
print(results)
print('SVM Training Accuracy : ',
      metrics.accuracy_score(y_train,
                             svc_classifier.predict(X_train))*100)
print('SVM Validation Accuracy : ',
      metrics.accuracy_score(y_test,
                             svc_classifier.predict(X_test))*100)

                 Model  Accuracy  Precision    Recall  F1 Score
0  Logistic Regression   0.87500   0.615385  0.347826  0.444444
1         SVM (Linear)   0.85625   0.000000  0.000000  0.000000
SVM Training Accuracy :  86.77390527256479
SVM Validation Accuracy :  85.625


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-12-77eef6c82ba5>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(model_results, ignore_index = True)


In [13]:
## Randomforest
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state = 10, n_estimators = 100,
                                    criterion = 'entropy')
rf_classifier.fit(X_train, y_train)

# Predicting Test Set
y_pred = rf_classifier.predict(X_test)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

rf_classifier_results = pd.DataFrame([['Random Forest (n=100)', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

results = results.append(rf_classifier_results, ignore_index = True)
print(results)
print('Random Forest Training Accuracy : ',
      metrics.accuracy_score(y_train,
                             rf_classifier.predict(X_train))*100)
print('Random Forest Validation Accuracy : ',
      metrics.accuracy_score(y_test,
                             rf_classifier.predict(X_test))*100)

                   Model  Accuracy  Precision    Recall  F1 Score
0    Logistic Regression   0.87500   0.615385  0.347826  0.444444
1           SVM (Linear)   0.85625   0.000000  0.000000  0.000000
2  Random Forest (n=100)   0.91250   0.846154  0.478261  0.611111
Random Forest Training Accuracy :  99.91063449508489
Random Forest Validation Accuracy :  91.25


<ipython-input-13-3253e2d5722f>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(rf_classifier_results, ignore_index = True)
